In [44]:
import warnings
warnings.filterwarnings('ignore')
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
import os
from utils import *

In [45]:
# call API keys and set environment variables
groc_api_key = get_groc_api_key()

# os.environ["MODEL_NAME"] = ''

In [46]:
serper_tool = SerperDevTool()
scraper_tool = ScrapeWebsiteTool()
linkdin = ScrapeWebsiteTool(
    website_url="https://www.linkedin.com/feed/"
)
youtube = ScrapeWebsiteTool(
    website_url="https://www.youtube.com/"
)

# Define tools


In [47]:
extractor = Agent(
    role = "User Data Extractor",
    goal = "Extract all accurate educational details done by {user} including certificates, projects,research, awards and skills.",
    backstory= "You are an advanced data collection specialist focused on "
                "gathering comprehensive educational and professional achievements of {user} "
                "Your expertise lies in extracting verified information about "
                "academic qualifications, certifications, notable projects, "
                "research publications, academic awards, and technical skills. "
                "You meticulously validate data from multiple sources to ensure "
                "accuracy and completeness of the educational profile. "
                "Your collected data serves as a foundation for creating "
                "detailed academic portfolios and professional profiles.",
    tools = [serper_tool, scraper_tool, linkdin, youtube],
    allow_delegation=False,
    verbose=True
)

In [48]:
project_extractor = Agent(
    role="Project Requirement Extractor",
    goal="Extract and analyze all project requirements and specifications from {project_docs} including technical needs, constraints, deliverables, and success criteria.",
    backstory="You are an expert requirements analyst specialized in "
              "dissecting project documentation to identify key requirements. "
              "Your expertise lies in extracting and categorizing both functional "
              "and non-functional requirements, technical specifications, "
              "project constraints, and success metrics. "
              "You meticulously analyze documentation from multiple sources "
              "to ensure comprehensive coverage of all project needs. "
              "Your extracted data serves as the foundation for "
              "project planning, resource allocation, and development roadmaps.",
    tools=[serper_tool, scraper_tool],
    # define a RAG tool for the documentation.
    allow_delegation=True,
    verbose=True
)

In [49]:
skill_analyzer = Agent(
    role="Skill Comparison Expert",
    goal="Compare and analyze the gap between required project skills and user's current skillset found in {extractor}",
    backstory="You are an expert skill analyst specialized in "
              "comparing project requirements against individual capabilities. "
              "Your expertise lies in identifying skill gaps found in {extractor}, "
              "matching competencies with project needs found in {project_extractor}, "
              "and providing detailed analysis of where skills align or fall short. "
              "You analyze both technical and soft skills, "
              "considering proficiency levels and experience depth. "
              "Your analysis helps in identifying training needs to accomplish the {project_extractor}, "
              "development opportunities, and potential project role assignments.",
    tools=[serper_tool, scraper_tool],
    allow_delegation=True,
    verbose=True
)

In [50]:
'''Learning Path Finder: Finds learning paths from LinkedIn Learning, YouTube, Coursera, Udemy, etc. This is essential and can be the Recommendation Agent, querying multiple APIs for resources.'''

learning_path_finder = Agent(
    role="Learning Path Recommender",
    goal="Find and recommend optimal learning paths and resources based on identified skill gaps and project requirements",
    backstory="You are an expert learning consultant specialized in "
              "creating personalized learning paths across multiple platforms. "
              "Your expertise lies in analyzing skill gaps identified by {skill_analyzer}, "
              "and recommending the most effective learning resources from "
              "LinkedIn Learning, YouTube, Coursera, Udemy, and other platforms. "
              "You consider factors like learning style, time constraints, "
              "cost effectiveness, and course ratings to create "
              "comprehensive learning paths that bridge skill gaps efficiently. "
              "You prioritize resources based on relevance, quality, and "
              "alignment with project requirements from {project_extractor}.",
    tools=[serper_tool, scraper_tool, linkdin, youtube],
    allow_delegation=True,
    verbose=True
)


In [51]:
Recommender = Agent(
    role="Best Path Recommender",
    goal="Research and recommend optimal paths by analyzing skill comparisons and project requirements",
    backstory="You are a career path optimization expert who combines "
              "deep learning path expertise with strategic career planning. "
              "Your role is to evaluate skill gaps from {skill_analyzer}, "
              "and not only recommend learning resources but also suggest "
              "the most efficient sequence and combination of skills to develop. "
              "You specialize in creating personalized development roadmaps "
              "that consider the user's current skills, project requirements from {project_extractor}, "
              "learning style preferences, and industry best practices. "
              "You prioritize recommendations based on impact, time-to-proficiency, "
              "and alignment with project goals, providing both immediate "
              "and long-term development strategies.",
    tools = [serper_tool, scraper_tool, linkdin, youtube],
    allow_delegation=False,
    verbose=True
)


In [52]:
data_store = Agent(
    role="Data Store Expert",
    goal="Store and manage all extracted data, learning paths, and recommendations in MongoDB while ensuring proper data organization and retrieval",
    backstory="You are a data management expert specialized in "
              "organizing and storing complex educational and project data. "
              "You receive outputs from the Extractor, Project Extractor, "
              "Skill Analyzer, and Learning Path Finder, storing them as "
              "structured documents in MongoDB. You ensure data consistency, "
              "implement proper indexing, and maintain relationships between "
              "different data components. You also prepare the data in a format "
              "suitable for UI presentation, working closely with other agents "
              "to maintain data flow and accessibility. Your expertise includes "
              "data validation, version control, and efficient querying patterns.",
    tools=[serper_tool, scraper_tool],
    allow_delegation=True,
    verbose=True
)

## Tasks

In [53]:
extract = Task(
    description=(
        "1. Analyze user data from extractor to identify existing "
            "educational background and skills.\n"
        "2. Review project requirements and specifications from "
            "project_extractor to understand technical needs.\n"
        "3. Compare user's current capabilities with project requirements "
            "to identify skill gaps.\n"
        "4. Develop a detailed learning path outline including "
            "prioritized skills to acquire and recommended resources.\n"
        "5. Consider time constraints and learning preferences "
            "when planning the development roadmap."
    ),
    expected_output="A comprehensive learning plan document "
        "detailing skill gaps analysis, prioritized learning objectives, "
        "and recommended learning resources.",
    agent=extractor,
)

In [54]:
analyze_project = Task(
    description=(
        "1. Extract and analyze all project requirements from provided documentation.\n"
        "2. Identify key technical specifications and constraints.\n"
        "3. List all required deliverables and success criteria.\n"
        "4. Document functional and non-functional requirements.\n"
        "5. Organize requirements by priority and dependencies."
    ),
    expected_output="A comprehensive project requirements document "
        "detailing technical specifications, constraints, deliverables, "
        "and success criteria in a structured format.",
    agent=project_extractor,
)


In [55]:
analyze_skills = Task(
    description=(
        "1. Review skill requirements extracted from project documentation.\n"
        "2. Analyze user's current skill profile and expertise levels.\n"
        "3. Perform detailed gap analysis between required and existing skills.\n"
        "4. Identify critical skill deficiencies that need immediate attention.\n"
        "5. Quantify proficiency levels needed vs. current levels.\n"
        "6. Prioritize skill gaps based on project criticality."
    ),
    expected_output="A comprehensive skill gap analysis report "
        "detailing required vs. existing skills, proficiency levels, "
        "and prioritized list of skills to develop.",
    agent=skill_analyzer
)


In [56]:
find_learning_path = Task(
    description=(
        "1. Review skill gaps identified by the skill analyzer.\n"
        "2. Search and evaluate learning resources across platforms "
            "(LinkedIn Learning, YouTube, Coursera, Udemy).\n"
        "3. Create personalized learning paths based on skill gaps "
            "and project requirements.\n"
        "4. Consider time constraints, learning style preferences, "
            "and cost effectiveness.\n"
        "5. Prioritize resources based on relevance, quality, and "
            "alignment with project needs.\n"
    ),
    expected_output="A comprehensive learning path document "
        "detailing recommended courses, tutorials, and resources "
        "across multiple platforms, organized by priority and "
        "estimated completion time.",
    agent=learning_path_finder
)

In [57]:
recommend_path = Task(
    description=(
        "1. Review skill gap analysis and learning paths identified.\n"
        "2. Evaluate effectiveness and efficiency of proposed learning paths.\n"
        "3. Develop strategic sequencing of skill development activities.\n"
        "4. Create personalized roadmap considering time-to-proficiency.\n"
        "5. Prioritize recommendations based on project impact and urgency.\n"
        "6. Provide both immediate and long-term development strategies."
    ),
    expected_output="A strategic development roadmap detailing "
        "optimized learning sequences, time estimates, and "
        "prioritized action items for both immediate and "
        "long-term skill development.",
    agent=Recommender
)


In [58]:
store_data = Task(
    description=(
        "1. Collect and validate data from all previous tasks and agents.\n"
        "2. Structure and organize the collected data for MongoDB storage.\n"
        "3. Store user profile data, project requirements, skill gaps, "
            "and learning paths.\n"
        "4. Implement proper indexing and data relationships.\n"
        "5. Prepare data in a format suitable for UI presentation.\n"
        "6. Ensure data consistency and maintain version control."
    ),
    expected_output="A confirmation of successful data storage with "
        "organized collections in MongoDB, including user profiles, "
        "project requirements, skill analyses, and learning paths, "
        "all properly indexed and structured for efficient retrieval.",
    agent=data_store
)

In [59]:
crew = Crew(
    agents=[extractor, project_extractor, skill_analyzer, learning_path_finder, Recommender, data_store],
    tasks=[extract, analyze_project, analyze_skills, find_learning_path, recommend_path, store_data],
    verbose=True
)

2025-03-02 21:45:03,818 - 31160 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [ ]:
# Set OpenAI API key

result = crew.kickoff(inputs={
    "user": "Diluksha Perera",  # Replace with actual user name
    "project_docs": """Project requirements:
    - Develop a machine learning model for predictive analytics
    - Use Python and TensorFlow
    - Implement data preprocessing and feature engineering
    - Create API endpoints for model deployment
    - Document code and model architecture""",  # Replace with actual project description
    "extractor": "",  # Empty string as placeholder for extractor's output
    "project_extractor": "",  # Empty string as placeholder for project_extractor's output
    "skill_analyzer": ""  # Empty string as placeholder for skill_analyzer's output
})

# Agent: User Data Extractor
## Task: 1. Analyze user data from extractor to identify existing educational background and skills.
2. Review project requirements and specifications from project_extractor to understand technical needs.
3. Compare user's current capabilities with project requirements to identify skill gaps.
4. Develop a detailed learning path outline including prioritized skills to acquire and recommended resources.
5. Consider time constraints and learning preferences when planning the development roadmap.
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for SerperDevToolSchema
search_query
  Field required [type=missing, input_value={'description': 'educatio...lls of Diluksha Perera'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: 

In [ ]:
formatted_output = '''
# Learning Path Analysis for Diluksha Perera

## User Profile
- **Name**: Diluksha Perera
- **Degree**: Bachelor of Science in Applied Data Science Communication
- **Institution**: General Sir John Kotelawala Defence University

### Current Skills
- Associate Data Engineer
- Strong programming and analytical skills  
- Data visualization and storytelling
- Data management and processing tools

### Certifications
- Microsoft Certified: Azure Fundamentals (2 certifications)
- Google Certified Data Analyst
- Generative AI Productivity Skills (Microsoft/LinkedIn)

## Project Requirements
### Functional Requirements
- Data Preprocessing
- Feature Engineering 
- Machine Learning Model Development
- API Endpoints
- User Interfaces

### Technical Specifications
- Programming Language: Python
- ML Framework: TensorFlow
- Deployment: Docker

## Skill Gap Analysis
### Major Gaps Identified
- Advanced Data Science Techniques
- Project Management Skills
- Big Data Technologies
- Feature Engineering
- API Development

## Recommended Learning Path
### Short Term (0-6 months)
1. Machine Learning by Andrew Ng (Coursera)
    - Duration: 11 weeks
    - Focus: Supervised/unsupervised learning

2. Project Management Principles and Practices (Coursera) 
    - Duration: 4 months
    - Focus: Project planning and execution

### Long Term (6-12 months)
1. Big Data Specialization (Coursera)
    - Duration: 4 months
    - Focus: Hadoop, Spark

2. Feature Engineering for Machine Learning
    - Duration: 3 hours
    - Focus: Feature selection and creation

### Additional Resources
- Books:
  - "Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow"
  - "Big Data: Principles and Best Practices"
- Activities:
  - Data science hackathons
  - Local meetups
  - Kaggle challenges
'''

from IPython.display import display, Markdown
display(Markdown(formatted_output))


# Learning Path Analysis for Diluksha Perera

## User Profile
- **Name**: Diluksha Perera
- **Degree**: Bachelor of Science in Applied Data Science Communication
- **Institution**: General Sir John Kotelawala Defence University

### Current Skills
- Associate Data Engineer
- Strong programming and analytical skills  
- Data visualization and storytelling
- Data management and processing tools

### Certifications
- Microsoft Certified: Azure Fundamentals (2 certifications)
- Google Certified Data Analyst
- Generative AI Productivity Skills (Microsoft/LinkedIn)

## Project Requirements
### Functional Requirements
- Data Preprocessing
- Feature Engineering 
- Machine Learning Model Development
- API Endpoints
- User Interfaces

### Technical Specifications
- Programming Language: Python
- ML Framework: TensorFlow
- Deployment: Docker

## Skill Gap Analysis
### Major Gaps Identified
- Advanced Data Science Techniques
- Project Management Skills
- Big Data Technologies
- Feature Engineering
- API Development

## Recommended Learning Path
### Short Term (0-6 months)
1. Machine Learning by Andrew Ng (Coursera)
    - Duration: 11 weeks
    - Focus: Supervised/unsupervised learning

2. Project Management Principles and Practices (Coursera) 
    - Duration: 4 months
    - Focus: Project planning and execution

### Long Term (6-12 months)
1. Big Data Specialization (Coursera)
    - Duration: 4 months
    - Focus: Hadoop, Spark

2. Feature Engineering for Machine Learning
    - Duration: 3 hours
    - Focus: Feature selection and creation

### Additional Resources
- Books:
  - "Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow"
  - "Big Data: Principles and Best Practices"
- Activities:
  - Data science hackathons
  - Local meetups
  - Kaggle challenges
